In [16]:
import pandas as pd
import statsmodels.api as sm

I need to add the possibility to add covariates to the untreated twfe--- check
I need to make sure I dont drop the treated but just separate them --- check
I need to construct the function that creates the aggrete parameters using proper weights, create a weights function for this purpose.
I need to make sure the graph for the hours over days is included in the data set 
test the boot error function, and make sure I get the diagonal of the covariance matrix.
I need to create another function that make the testing of parallel trends on the untreated

In [17]:
def compute_treatment_effects(data, outcome, treatment, unit_id, time_id, covariates=[]):
    # Filter the untreated units for model estimation
    data_untreated = data[data[treatment] == 0]
    
    if len(data_untreated) == 0:
        raise ValueError("There are no untreated units for comparison.")
    
    # Prepare the variables for fixed-effects regression
    features = [outcome, treatment] + covariates
    X = data_untreated.drop(features, axis=1)
    X = pd.get_dummies(X, columns=[unit_id, time_id], drop_first=True)
    
    # Convert boolean columns to integers
    X = X.astype({col: 'int64' for col in X.select_dtypes(include=['bool']).columns})
    
    # Add covariates if any
    if covariates:
        X[covariates] = data_untreated[covariates]
        
    y = data_untreated[outcome]
    
    # Run the two-way fixed effects regression on untreated units
    model = sm.OLS(y, sm.add_constant(X))
    results = model.fit()
    
    # Create a column of predicted values only for treated units
    data_treated = data[data[treatment] == 1]
    X_treated = data_treated.drop(features, axis=1)
    X_treated = pd.get_dummies(X_treated, columns=[unit_id, time_id], drop_first=True)
    
    # Convert boolean columns to integers
    X_treated = X_treated.astype({col: 'int64' for col in X_treated.select_dtypes(include=['bool']).columns})
    
    # Add covariates if any
    if covariates:
        X_treated[covariates] = data_treated[covariates]
        
    data.loc[data[treatment] == 1, 'y_0'] = results.predict(sm.add_constant(X_treated))
    
    # Compute treatment effects
    data['t_effects'] = data[outcome] - data['y_0']
    
    return data

In [18]:
# Create a toy dataset as an example
import numpy as np
# Number of units and time periods
n_units = 10
n_time = 10

# Generate a DataFrame
np.random.seed(42)  # For reproducibility
df = pd.DataFrame({
    'unit_id': np.repeat(range(1, n_units + 1), n_time),
    'time_id': list(range(1, n_time + 1)) * n_units,
    'treatment': np.random.choice([0, 1], n_units * n_time),
    'covariate1': np.random.normal(0, 1, n_units * n_time),
    'covariate2': np.random.normal(0, 1, n_units * n_time),
})

# Simulating potential outcomes under the control (Y0)
# Using random normal values for simplicity
df['Y0'] = 5 + 0.5 * df['covariate1'] + 0.3 * df['covariate2'] + \
          0.2 * df['unit_id'] + 0.1 * df['time_id'] + np.random.normal(0, 1, n_units * n_time)

# Defining treatment effect (constant for all units as 2)
df['treatment_effect'] = 2

# Simulating potential outcomes under the treatment (Y1)
df['Y1'] = df['Y0'] + df['treatment_effect']

# Constructing the observed outcome based on treatment status
df['outcome'] = np.where(df['treatment'] == 1, df['Y1'], df['Y0'])

In [19]:
df.head()

,unit_id,time_id,treatment,covariate1,covariate2,Y0,treatment_effect,Y1,outcome
0,1,1,0,0.738467,0.227460,4.944951,2,6.944951,4.944951
1,1,2,1,0.171368,1.307143,5.763091,2,7.763091,7.763091
2,1,3,0,-0.115648,-1.607483,5.464918,2,7.464918,5.464918
3,1,4,0,-0.301104,0.184634,6.370594,2,8.370594,6.370594
4,1,5,0,-1.478522,0.259883,3.838407,2,5.838407,3.838407


In [20]:
# Apply the function
result_df = compute_treatment_effects(df, 'outcome', 'treatment', 'unit_id', 'time_id', covariates=['covariate1', 'covariate2'])
result_df.head()

,unit_id,time_id,treatment,covariate1,covariate2,Y0,treatment_effect,Y1,outcome,y_0,t_effects
0,1,1,0,0.738467,0.227460,4.944951,2,6.944951,4.944951,NaN,NaN
1,1,2,1,0.171368,1.307143,5.763091,2,7.763091,7.763091,5.763091,2.0
2,1,3,0,-0.115648,-1.607483,5.464918,2,7.464918,5.464918,NaN,NaN
3,1,4,0,-0.301104,0.184634,6.370594,2,8.370594,6.370594,NaN,NaN
4,1,5,0,-1.478522,0.259883,3.838407,2,5.838407,3.838407,NaN,NaN


In [21]:
result_df.loc[result_df['treatment']==1].head()

,unit_id,time_id,treatment,covariate1,covariate2,Y0,treatment_effect,Y1,outcome,y_0,t_effects
1,1,2,1,0.171368,1.307143,5.763091,2,7.763091,7.763091,5.763091,2.0
5,1,6,1,-0.719844,0.781823,5.340124,2,7.340124,7.340124,5.340124,2.0
9,1,10,1,-1.763040,0.296985,5.812557,2,7.812557,7.812557,5.812557,2.0
14,2,5,1,1.031000,0.293072,4.984052,2,6.984052,6.984052,4.984052,2.0
16,2,7,1,-0.839218,1.865775,7.507035,2,9.507035,9.507035,7.507035,2.0
